In [1]:
import numpy as np 
import py2neo as pn 
import random as rd
import math
import copy

In [34]:
def barabasi(total_n, starting_n):
    ### Generate a Barabasi-Albert network simulation
    # total_n: total number of nodes
    # starting_n: number of nodes to generate edges uniformly at random
    # returns subgraph: neo4j graph object
    ###
    edges = []
    nodes = []
    for i in range(starting_n):
        # Generate nodes
        temp_nodes = [pn.Node('User', name='user' + str(i)) for i in range(starting_n)]
        nodes.extend(temp_nodes)
        # Generate links uniformly at random
        temp = [pn.Relationship(nodes[i], "follow", nodes[j]) for j in range(starting_n)
                       if rd.uniform(0,1) < 1/2 and i != j]
        edges.extend(temp)
        temp_subgraph = pn.Subgraph(nodes,edges)
    for l in range(starting_n, total_n):
        nodes.append(pn.Node('User', name='user' + str(l)))
        n = len(edges)
        # Compute the probability for a new node to connect to each existing node
        temp_probs = [len(list(filter(lambda x: x.end_node == nodes[j], relationships))) for j in range(len(nodes))]
        # Generate links for a new node
        temp = [pn.Relationship(nodes[l], "follow", nodes[j]) for j in range(l) 
                        if rd.uniform(0,1) < temp_probs[j] and l != j]
        edges.extend(temp)
        temp_subgraph = pn.Subgraph(nodes,edges)
    subgraph = pn.Subgraph(nodes,edges)
    return subgraph

In [31]:
def postpublish(graph, m0, m1):
    ### Simulate a list of posts and users who create them
    # graph: py2neo object for follower network
    # m0: starting number of posts
    # m1: total number of posts
    # returns postpublished: list of tuples (i,j) where user i posted post j
    ###
    postpublished = []
    
    # Get the number of nodes
    n = len(graph.nodes)
    
    # Generate the first posts
    for i in range(m0):
        postpublished.append(pn.Node('Post', name='post' + str(i)))
    
    # Generate the rest of the posts
    for m in range(m0,m1):
        # Compute the prob that anyone posts
        probs = []
        for k in range(0,n):
            temp_probs = [len(list(filter(lambda x: j == x[1], friends_edges)))
                                  for (i,j) in friends_edges]
            temp_probs = temp_probs + [len(list(filter(lambda x: j == x[1], follower_edges)))
                                  for (i,j) in follower_edges]
            temp_probs = temp_probs + [len(list(filter(lambda x: i == x[0], postpublished)))
                                  for (i,j) in postpublished]
            probs.append(temp_probs)
        temp_user = rd.choices(list[range(0,n)], weights = probs)
        postpublished.append((temp_user,m))                      
    
    return postpublished
    

In [9]:
    # start a new project in Neo4j and set connections
    graph = pn.Graph(
        host = 'localhost',
        http_port = '7474',
        user = 'neo4j',
        password = '2500'
        )   

In [36]:
followers = barabasi(100,5)
graph.run('match (n:User) detach delete n')
graph.create(followers)

In [13]:
nodes = [pn.Node('User', name='user' + str(i)) for i in range(5)]
nodes

[Node('User', name='user0'),
 Node('User', name='user1'),
 Node('User', name='user2'),
 Node('User', name='user3'),
 Node('User', name='user4')]

In [17]:
relationships = [pn.Relationship(nodes[i], "follow", nodes[rd.choice(range(5))]) for i in range(5)]

In [18]:
relationships

[follow(Node('User', name='user0'), Node('User', name='user3')),
 follow(Node('User', name='user1'), Node('User', name='user4')),
 follow(Node('User', name='user2'), Node('User', name='user2')),
 follow(Node('User', name='user3'), Node('User', name='user3')),
 follow(Node('User', name='user4'), Node('User', name='user2'))]

In [20]:
relationships[1].start_node

Node('User', name='user1')

In [31]:
una_lista = filter(lambda x: x.end_node == nodes[3], relationships)

In [32]:
len(list(una_lista))

2

In [33]:
nodes[4]

Node('User', name='user4')